<a id="title_ID"></a>
# JWST Pipeline Validation Testing Notebook: spec2, pathloss step

<span style="color:red"> **Instruments Affected**</span>: NIRSpec 

Tested on CV3 data

### Table of Contents
<div style="text-align: left"> 

<br> [Imports](#imports_ID) <br> [Introduction](#intro_ID) <br> [Testing Data Set](#data_ID) <br> [Run the JWST pipeline and assign_wcs validation tests](#pipeline_ID): [FS Full-Frame test](#FULLFRAME), [FS ALLSLITS test](#ALLSLITS), [MOS test](#MOS) <br> [About This Notebook](#about_ID)<br> [Results](#results) <br>

</div>

<a id="imports_ID"></a>
# Imports
The library imports relevant to this notebook are aready taken care of by importing PTT.

* astropy.io for opening fits files
* jwst.module.PipelineStep is the pipeline step being tested
* matplotlib.pyplot.plt to generate plot

NOTE: This notebook assumes that the pipeline version to be tested is already installed and its environment is activated.

To be able to run this notebook you need to install nptt. 

If all goes well you will be able to import PTT.

[Top of Page](#title_ID)

In [1]:
import os
from astropy.io import fits

import jwst
from jwst.pathloss import PathLossStep

# The latest version of NPTT is installed in the requirements text file at:
# /jwst_validation_notebooks/requirements.txt

# import NPTT
import nirspec_pipe_testing_tool as nptt

In [2]:
# VERSIONS

# print the jwst pipeline version
print("Using JWST pipeline version: ", jwst.__version__)

# Make sure that the version used is the right one

# to get the latest version from 'master' run:
# subprocess.run(['pip', 'install', 'git+https://github.com/spacetelescope/nirspec_pipe_testing_tool@master'])

latest_nptt_stable_version = '1.1.1'
nptt_version = nptt.__version__

if nptt_version != latest_nptt_stable_version:
    print("\n* WARNING - The code might break because the version used is not the latest stable version:")
    print("            Latest stable version is ", latest_nptt_stable_version)
    print("            Using NPTT version ", nptt_version)

Using JWST pipeline version:  0.16.2

* WARNING - The code might break because the version used is not the latest stable version:
            Latest stable version is  1.1.1
            Using NPTT version  1.1.1.dev4+gb2081e9


<a id="intro_ID"></a>
# Introduction

The NIRSpec team developed an implementation of the pathloss step and compared the output to the pipeline output for this step. We expected machine precision differences (about 10^-7) but due to rounding differences in interpolation methods we relaxed the difference threshold to 10^-6. 

For the test to be considered PASSED, every single slit (for FS data), slitlet (for MOS data) or slice (for IFU data) in the input file has to pass. If there is any failure, the whole test will be considered as FAILED. 

The code for this test for Fixed Slits (FS) can be obtained from: https://github.com/spacetelescope/nirspec_pipe_testing_tool/blob/master/nirspec_pipe_testing_tool/calwebb_spec2_pytests/auxiliary_code/pathloss_fs_ps.py. There is a corresponding script for Multi Object Spectroscopy (MOS) and for Integral Field Unit (IFU) data. 


Step description: https://jwst-pipeline.readthedocs.io/en/latest/jwst/pathloss/description.html

Pipeline code: https://github.com/spacetelescope/jwst/tree/master/jwst/pathloss


### Calibration WG Requested Algorithm: 

A short description and link to the page: 
https://outerspace.stsci.edu/display/JWSTCC/Vanilla+Path-Loss+Correction


### Defining Term
Acronymns used un this notebook:

pipeline: calibration pipeline

spec2: spectroscopic calibration pipeline level 2b

PTT: NIRSpec pipeline testing tool (https://github.com/spacetelescope/nirspec_pipe_testing_tool)


[Top of Page](#title_ID)

<a id="data_ID"></a>
# Testing Data Set

All testing data is from the CV3 campaign. We chose these files because this is our most complete data set, i.e. all modes and filter-grating combinations.

Data used was for testing was only FS and MOS, since extract_2d is skipped for IFU. Data sets are:
- FS_FULLFRAME_G395H_F290LP
- FS_ALLSLITS_G140H_F100LP 
- MOS_G140M_LINE1 

The testing data can be located at ```/grp/jwst/wit4/nirspec_vault/prelaunch_data/testing_sets/b7.1_pipeline_testing/test_data_suite```

In that directory you will find a subdirectory for CV3 data for FS, MOS, and IFU, as well as other campaigns. This notebook is specifically written for CV3 data.



[Top of Page](#title_ID)

Set the following variables to the file you are interested in testing:

- output_directory = path where you want intermediary files and plots to be saved in

- input_file = this is the count rate file, e.g. jwtest1_NRS1_rate.fits

- mode_used = supported modes are: FS, MOS, IFU, BOTS, MOS_sim, dark, image, confirm, taconfirm, wata, msata, focus, mimf

- raw_data_root_file = CV3 raw data file


<a id="pipeline_ID"></a>
# Run the JWST pipeline and extract_2d validation tests

The pipeline can be run from the command line in two variants: full or per step.

Tu run the spec2 pipeline in full use the command: 

$ strun jwst.pipeline.Spec2Pipeline jwtest_rate.fits

Tu only run the extract_2d step, use the command:

$ strun jwst.extract_2d.Extract2dStep jwtest_previous_step_output.fits


These options are also callable from a script with the testing environment active. The Python call for running the pipeline in full or by step are:

$\gt$ from jwst.pipeline.calwebb_spec2 import Spec2Pipeline

$\gt$ Spec2Pipeline.call(jwtest_rate.fits)
 
or
 
$\gt$ from jwst.extract_2d import Extract2dStep
 
$\gt$ Extract2dStep.call(jwtest_previous_step_output.fits)


PTT can run the spec2 pipeline either in full or per step, as well as the imaging pipeline in full. In this notebook we will use PTT to run the pipeline and the validation tests. To run PTT, follow the directions in the corresponding repo page.

[Top of Page](#title_ID)

-> For each mode, the following variables will need to be set:

- output_directory = path where you want intermediary files and plots to be saved in

- input_file = this is the count rate file, e.g. jwtest1_NRS1_rate.fits

- mode_used = supported modes are: FS, MOS, IFU, BOTS, MOS_sim, dark, image, confirm, taconfirm, wata, msata, focus, mimf

- raw_data_root_file = CV3 raw data file


<a id="Common_variables"></a>
# Common Variables

In [6]:
# Set common variables for entire notebook

# paths
vault_path = os.path.join(os.environ.get('WIT4_PATH'), 'nirspec_vault')
esa_files_path = os.path.join(vault_path, 'prelaunch_data/testing_sets/b7.1_pipeline_testing/test_data_suite/')
test_data_path = '/Users/pena/Documents/PyCharmProjects/nirspec/pipeline/testing_data'

# step to run
run_calwebb_spec2 = 'extract_2d'

# accepted threshold difference with respect to benchmark files
extract_2d_threshold_diff = 4

# define benchmark (or 'truth') file
compare_assign_wcs_and_extract_2d_with_esa = False


<a id="FULLFRAME"></a>
# FS FULL-FRAME test

In [7]:
# Set the variables for Fixed Slit (FS) data
data_directory = os.path.join(test_data_path, 'FS_FULL_FRAME/G395H_opaque')
input_file = os.path.join(data_directory, 'final_output_caldet1_NRS1_assign_wcs.fits')
mode_used = 'FS'
output_directory = 'somewhere_to_save_the_plots'

In [8]:
# ESA intermediary products
raw_data_root_file = 'NRSSMOS-MOD-G3H-02-5344120942_1_491_SE_2015-12-10T12h18m25.fits'
esa_files_full_path = "".join([esa_files_path, mode_used, "_CV3_cutouts/ESA_Int_products"])

# pipeline intermediary products
truth_file = os.path.join(data_directory, 'final_output_caldet1_NRS1_extract_2d_truth.fits')

if not compare_assign_wcs_and_extract_2d_with_esa:
    esa_files_path, raw_data_root_file = None, None
else:
    truth_file = None

In [5]:
# Run the pipeline step
step_output_file = input_file.replace(".fits", "_extract_2d.fits")
pipe_result = Extract2dStep.call(input_file)
#pipe_result.save(step_output_file)   # this is to save the pipeline output to a file

2020-09-24 17:39:42,089 - CRDS - ERROR -  Error determining best reference for 'pars-extract2dstep'  =   Unknown reference type 'pars-extract2dstep'
2020-09-24 17:39:42,092 - stpipe.Extract2dStep - INFO - Extract2dStep instance created.
2020-09-24 17:39:42,173 - stpipe.Extract2dStep - INFO - Step Extract2dStep running with args ('/Users/pena/Documents/PyCharmProjects/nirspec/pipeline/testing_data/FS_FULL_FRAME/G395H_opaque/final_output_caldet1_NRS1_assign_wcs.fits',).
2020-09-24 17:39:42,175 - stpipe.Extract2dStep - INFO - Step Extract2dStep parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '', 'slit_name': None, 'apply_wavecorr': True, 'extract_orders': None, 'extract_height': None, 'grism_objects': None, 'mmag_extract': 99.0}
2020-09-24 17:39:46,196 - stpipe.Extract2dStep -

In [6]:
# Run the validation test
%matplotlib inline
result, _ = nptt.calwebb_spec2_pytests.auxiliary_code.check_corners_extract2d.find_FSwindowcorners(pipe_result, 
                                                              truth_file=truth_file,
                                                              esa_files_path=esa_files_path,
                                                              extract_2d_threshold_diff=extract_2d_threshold_diff)

Information from the 'truth' (or comparison) file 
Filename: /Users/pena/Documents/PyCharmProjects/nirspec/pipeline/testing_data/FS_FULL_FRAME/G395H_opaque/final_output_caldet1_NRS1_extract_2d_truth.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     278   ()      
  1  SCI           1 ImageHDU        57   (1225, 62)   float32   
  2  DQ            1 ImageHDU        11   (1225, 62)   int32 (rescales to uint32)   
  3  ERR           1 ImageHDU        10   (1225, 62)   float32   
  4  WAVELENGTH    1 ImageHDU         9   (1225, 62)   float32   
  5  VAR_POISSON    1 ImageHDU         9   (1225, 62)   float32   
  6  VAR_RNOISE    1 ImageHDU         9   (1225, 62)   float32   
  7  SCI           2 ImageHDU        56   (1415, 58)   float32   
  8  DQ            2 ImageHDU        11   (1415, 58)   int32 (rescales to uint32)   
  9  ERR           2 ImageHDU        10   (1415, 58)   float32   
 10  WAVELENGTH    2 ImageHDU         9   (1415, 5

In [7]:
# Quickly see if the test passed 
   
print("Did assign_wcs validation for FS FULL-FRAME test passed? ", result)


Did assign_wcs validation for FS FULL-FRAME test passed?  True


<a id="ALLSLITS"></a>
# FS ALLSLITS test

In [8]:
# Set the variables for Fixed Slit (FS) data
data_directory = os.path.join(test_data_path, 'FS_ALLSLITS/G140H_F100LP')
input_file = os.path.join(data_directory, 'final_output_caldet1_NRS1_F100LP_assign_wcs.fits')
mode_used = 'FS'
output_directory = 'somewhere_to_save_the_plots'

In [9]:
# ESA intermediary products
raw_data_root_file = 'NRSSDRK-ALLSLITS-5345150216_1_491_SE_2015-12-11T15h40m25.fits'
esa_files_full_path = "".join([esa_files_path, mode_used, "_CV3_cutouts/ESA_Int_products"])

# pipeline intermediary products
truth_file = os.path.join(data_directory, 'final_output_caldet1_NRS1_F100LP_extract_2d_truth.fits')

if not compare_assign_wcs_and_extract_2d_with_esa:
    esa_files_path, raw_data_root_file = None, None
else:
    truth_file = None

In [10]:
# Run the pipeline step
step_output_file = input_file.replace(".fits", "_extract_2d.fits")
pipe_result = Extract2dStep.call(input_file)
#pipe_result.save(step_output_file)   # this is to save the pipeline output to a file

2020-09-24 17:39:53,884 - CRDS - ERROR -  Error determining best reference for 'pars-extract2dstep'  =   Unknown reference type 'pars-extract2dstep'
2020-09-24 17:39:53,886 - stpipe.Extract2dStep - INFO - Extract2dStep instance created.
2020-09-24 17:39:53,988 - stpipe.Extract2dStep - INFO - Step Extract2dStep running with args ('/Users/pena/Documents/PyCharmProjects/nirspec/pipeline/testing_data/FS_ALLSLITS/G140H_F100LP/final_output_caldet1_NRS1_F100LP_assign_wcs.fits',).
2020-09-24 17:39:53,990 - stpipe.Extract2dStep - INFO - Step Extract2dStep parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '', 'slit_name': None, 'apply_wavecorr': True, 'extract_orders': None, 'extract_height': None, 'grism_objects': None, 'mmag_extract': 99.0}
2020-09-24 17:39:57,461 - stpipe.Extract2dS

In [11]:
# Run the validation test
%matplotlib inline
result, _ = nptt.calwebb_spec2_pytests.auxiliary_code.check_corners_extract2d.find_FSwindowcorners(pipe_result, 
                                                              truth_file=truth_file,
                                                              esa_files_path=esa_files_path,
                                                              extract_2d_threshold_diff=extract_2d_threshold_diff)

Information from the 'truth' (or comparison) file 
Filename: /Users/pena/Documents/PyCharmProjects/nirspec/pipeline/testing_data/FS_ALLSLITS/G140H_F100LP/final_output_caldet1_NRS1_F100LP_extract_2d_truth.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     287   ()      
  1  SCI           1 ImageHDU        56   (1403, 64)   float32   
  2  DQ            1 ImageHDU        11   (1403, 64)   int32 (rescales to uint32)   
  3  ERR           1 ImageHDU        10   (1403, 64)   float32   
  4  WAVELENGTH    1 ImageHDU         9   (1403, 64)   float32   
  5  VAR_POISSON    1 ImageHDU         9   (1403, 64)   float32   
  6  VAR_RNOISE    1 ImageHDU         9   (1403, 64)   float32   
  7  SCI           2 ImageHDU        56   (1593, 60)   float32   
  8  DQ            2 ImageHDU        11   (1593, 60)   int32 (rescales to uint32)   
  9  ERR           2 ImageHDU        10   (1593, 60)   float32   
 10  WAVELENGTH    2 ImageHDU         9   (15

2020-09-24 17:40:02,849 - stpipe - WARNING - /Users/pena/anaconda3/envs/jwst162/lib/python3.7/site-packages/jwst/datamodels/validate.py:35: ValidationWarning: While validating meta.instrument.fixed_slit the following error occurred:
'ALLSLITS' is not one of ['S200A1', 'S200A2', 'S200B1', 'S400A1', 'S1600A1', 'NONE']

Failed validating 'enum' in schema:
    {'$schema': 'http://stsci.edu/schemas/asdf-schema/0.1.0/asdf-schema',
     'blend_table': True,
     'enum': ['S200A1', 'S200A2', 'S200B1', 'S400A1', 'S1600A1', 'NONE'],
     'fits_keyword': 'FXD_SLIT',
     'title': 'Name of fixed slit aperture used',
     'type': 'string'}

On instance:
    'ALLSLITS'
  warnings.warn(errmsg, ValidationWarning)



from assign_wcs file  -->     Detector: NRS1   Grating: G140H   Filter: F100LP   Lamp: NO_LAMP

Working with slit: S200A1
    Truth slit size =  [1403, 64]
 Pipeline slit size =  1403 64
Corners for slit S200A1:  [x0, y0, x1, y1]
   Truth corners: [646, 1056, 2049, 1120]
    Pipeline corners: [646, 1056, 2049, 1120]
* Test PASSED: All corners match within the threshold.

Working with slit: S200A2
    Truth slit size =  [1593, 60]
 Pipeline slit size =  1593 60
Corners for slit S200A2:  [x0, y0, x1, y1]
   Truth corners: [456, 1019, 2049, 1079]
    Pipeline corners: [456, 1019, 2049, 1079]
* Test PASSED: All corners match within the threshold.

Working with slit: S400A1
    Truth slit size =  [1475, 67]
 Pipeline slit size =  1475 67
Corners for slit S400A1:  [x0, y0, x1, y1]
   Truth corners: [574, 979, 2049, 1046]
    Pipeline corners: [574, 979, 2049, 1046]
* Test PASSED: All corners match within the threshold.

Working with slit: S1600A1
    Truth slit size =  [1452, 45]
 Pipeline s

In [12]:
# Quickly see if the test passed 
   
print("Did assign_wcs validation test for FS ALLSLITS passed? ", result)


Did assign_wcs validation test for FS ALLSLITS passed?  True


<a id="MOS"></a>
# MOS test

In [19]:
# Set the variables 
data_directory = os.path.join(test_data_path, 'MOS/G140M_LINE1')
input_file = os.path.join(data_directory, 'final_output_caldet1_NRS1_msa_flagging.fits')
mode_used = 'MOS'
msa_conf_name = os.path.join(data_directory,'V8460001000101_msa.fits')
output_directory = 'somewhere_to_save_the_plots'

In [20]:
# ESA intermediary products
raw_data_root_file = 'NRSV84600010001P0000000002101_4_491_SE_2016-01-17T17h34m08.fits'
esa_files_full_path = "".join([esa_files_path, mode_used, "_CV3_cutouts/ESA_Int_products"])

# pipeline intermediary products
truth_file = os.path.join(data_directory, 'final_output_caldet1_NRS1_extract_2d_truth.fits')

if not compare_assign_wcs_and_extract_2d_with_esa:
    esa_files_path, raw_data_root_file = None, None
else:
    truth_file = None

In [21]:
# Run the pipeline step
step_output_file = input_file.replace(".fits", "_extract_2d.fits")
pipe_result = Extract2dStep.call(input_file)
#pipe_result.save(step_output_file)   # this is to save the pipeline output to a file

2020-09-25 08:33:29,536 - CRDS - ERROR -  Error determining best reference for 'pars-extract2dstep'  =   Unknown reference type 'pars-extract2dstep'
2020-09-25 08:33:29,538 - stpipe.Extract2dStep - INFO - Extract2dStep instance created.
2020-09-25 08:33:29,959 - stpipe.Extract2dStep - INFO - Step Extract2dStep running with args ('/Users/pena/Documents/PyCharmProjects/nirspec/pipeline/testing_data/MOS/G140M_LINE1/final_output_caldet1_NRS1_msa_flagging.fits',).
2020-09-25 08:33:29,960 - stpipe.Extract2dStep - INFO - Step Extract2dStep parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '', 'slit_name': None, 'apply_wavecorr': True, 'extract_orders': None, 'extract_height': None, 'grism_objects': None, 'mmag_extract': 99.0}
2020-09-25 08:33:49,113 - stpipe.Extract2dStep - INFO - E

2020-09-25 08:34:02,159 - stpipe.Extract2dStep - INFO - Subarray y-extents are: 202 218
2020-09-25 08:34:02,208 - stpipe.Extract2dStep - INFO - Input model type is ImageModel
2020-09-25 08:34:02,290 - stpipe.Extract2dStep - INFO - Detected an extended source in slit 46, stellarity is 0.0
2020-09-25 08:34:02,290 - stpipe.Extract2dStep - INFO - Slit 46: Wavelength zero-point correction was not applied.
2020-09-25 08:34:02,292 - stpipe.Extract2dStep - INFO - slit.ymin -0.6000000000000001
2020-09-25 08:34:02,294 - stpipe.Extract2dStep - INFO - set slit_attributes completed
2020-09-25 08:34:02,306 - stpipe.Extract2dStep - INFO - Update S_REGION to POLYGON ICRS  156.211127421 -45.641279813 156.211000463 -45.641159678 156.210938572 -45.641191268 156.211065527 -45.641311404
2020-09-25 08:34:02,307 - stpipe.Extract2dStep - INFO - extract_2d updated S_REGION to POLYGON ICRS  156.211127421 -45.641279813 156.211000463 -45.641159678 156.210938572 -45.641191268 156.211065527 -45.641311404
2020-09-25

2020-09-25 08:34:15,403 - stpipe.Extract2dStep - INFO - Input model type is ImageModel
2020-09-25 08:34:15,489 - stpipe.Extract2dStep - INFO - Detected an extended source in slit 55, stellarity is 0.0
2020-09-25 08:34:15,489 - stpipe.Extract2dStep - INFO - Slit 55: Wavelength zero-point correction was not applied.
2020-09-25 08:34:15,491 - stpipe.Extract2dStep - INFO - slit.ymin -0.6000000000000001
2020-09-25 08:34:15,494 - stpipe.Extract2dStep - INFO - set slit_attributes completed
2020-09-25 08:34:15,506 - stpipe.Extract2dStep - INFO - Update S_REGION to POLYGON ICRS  156.211226719 -45.684941334 156.211098073 -45.684818233 156.211035880 -45.684850241 156.211164524 -45.684973343
2020-09-25 08:34:15,507 - stpipe.Extract2dStep - INFO - extract_2d updated S_REGION to POLYGON ICRS  156.211226719 -45.684941334 156.211098073 -45.684818233 156.211035880 -45.684850241 156.211164524 -45.684973343
2020-09-25 08:34:17,410 - stpipe.Extract2dStep - INFO - Name of subarray extracted: 65
2020-09-25 

2020-09-25 08:34:29,164 - stpipe.Extract2dStep - INFO - Detected an extended source in slit 59, stellarity is 0.0
2020-09-25 08:34:29,165 - stpipe.Extract2dStep - INFO - Slit 59: Wavelength zero-point correction was not applied.
2020-09-25 08:34:29,167 - stpipe.Extract2dStep - INFO - slit.ymin -0.6000000000000001
2020-09-25 08:34:29,169 - stpipe.Extract2dStep - INFO - set slit_attributes completed
2020-09-25 08:34:29,182 - stpipe.Extract2dStep - INFO - Update S_REGION to POLYGON ICRS  156.167987333 -45.658726530 156.167861814 -45.658605973 156.167800128 -45.658637729 156.167925645 -45.658758288
2020-09-25 08:34:29,183 - stpipe.Extract2dStep - INFO - extract_2d updated S_REGION to POLYGON ICRS  156.167987333 -45.658726530 156.167861814 -45.658605973 156.167800128 -45.658637729 156.167925645 -45.658758288
2020-09-25 08:34:30,589 - stpipe.Extract2dStep - INFO - Name of subarray extracted: 60
2020-09-25 08:34:30,589 - stpipe.Extract2dStep - INFO - Subarray x-extents are: 641 2048
2020-09-2

In [ ]:
# Run the validation test
%matplotlib inline
mos_result, _ = nptt.calwebb_spec2_pytests.auxiliary_code.check_corners_extract2d.find_MOSwindowcorners(pipe_result,
                                                               msa_conf_name,
                                                               truth_file=truth_file,
                                                               esa_files_path=esa_files_path,
                                                               extract_2d_threshold_diff= extract_2d_threshold_diff)

Information from the 'truth' (or comparison) file 
Filename: /Users/pena/Documents/PyCharmProjects/nirspec/pipeline/testing_data/MOS/G140M_LINE1/final_output_caldet1_NRS1_extract_2d_truth.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     275   ()      
  1  SCI           1 ImageHDU        66   (506, 14)   float32   
  2  DQ            1 ImageHDU        11   (506, 14)   int32 (rescales to uint32)   
  3  ERR           1 ImageHDU        10   (506, 14)   float32   
  4  WAVELENGTH    1 ImageHDU         9   (506, 14)   float32   
  5  VAR_POISSON    1 ImageHDU         9   (506, 14)   float32   
  6  VAR_RNOISE    1 ImageHDU         9   (506, 14)   float32   
  7  SCI           2 ImageHDU        66   (506, 14)   float32   
  8  DQ            2 ImageHDU        11   (506, 14)   int32 (rescales to uint32)   
  9  ERR           2 ImageHDU        10   (506, 14)   float32   
 10  WAVELENGTH    2 ImageHDU         9   (506, 14)   float32   
 11  

None


In [ ]:
# Quickly see if the test passed 
   
print("Did assign_wcs validation for MOS test passed? ", mos_result)


<a id="results"></a>
## Results

If the test **PASSED** this means that all slits, slitlets, or slices individually passed the test. However, if ony one individual slit (for FS data), slitlet (for MOS data) or slice (for IFU data) test failed, the whole test will be reported as **FAILED**.

<a id="about_ID"></a>
## About this Notebook
**Author:** Maria A. Pena-Guerrero, Staff Scientist II - Systems Science Support, NIRSpec
<br>**Updated On:** 04/09/2020

[Top of Page](#title_ID)
<img style="float: right;" src="./stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="stsci_pri_combo_mark_horizonal_white_bkgd" width="200px"/> 